# Paso a paso de la ejecucion del script
En este nootebook se irán explicando paso a paso las partes creadas para que funcine nuestro código

In [ ]:
import pandas as pd
import numpy as np
import re
from joblib import load, dump
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, GridSearchCV, TimeSeriesSplit
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.compose import TransformedTargetRegressor

## Lectura del dataset

In [ ]:
# Lectura del dataset
fullDataset = pd.read_csv('../datasets/data_target_stv_2017.csv', index_col=0, parse_dates=True)
x_col, target_col = fullDataset.columns[:-1], fullDataset.columns[-1]
data, target = fullDataset[x_col], fullDataset[target_col]

## Parte optativa de reducción de dimension
Esta parte es para las pruebas iniciales, luego se debe omitir, puesto que estamos reduciendo el dataset

In [ ]:
### En este caso el dataset ya está en el intervalo [0-1]
### En caso de que no lo estuviera, dividimos entre la potencia máxima del parque
 
#pot_install = 17560
#target = target / pot_install
### Valores desde los que crear la nueva región
midLat = 43.375
midLon = -7.875
jump = 0.125
n = 5
### Latitud y longitud
listLat = list(np.arange(midLat-jump*n, midLat+jump*(n+1), jump))
listLon = list(np.arange(midLon-jump*n, midLon+jump*(n+1), jump))
search_coord = []
for lat in listLat:
    for lon in listLon:
        search_coord.append(f'({lat}, {lon})')
### Con una expresión regular limpiamos las columnas que no nos interesna
col = list(data.columns)
new_coord = []
for coord in search_coord:
    r = re.compile(f".*{coord}")
    new_coord += list(filter(r.match, col))
newDataframe = data[new_coord]
newDataframe.index = data.index
data = newDataframe

## Partes a incluir en el pipeline

In [ ]:
# Creación de PCA sin parámetros para luego hiperparametrizar
pca = PCA()
# Creación del modelo de regresion final
reg = Ridge(max_iter=10000)

## Creación del Pipeline y estandarización del target

In [ ]:
# Pipeline
regr_base = Pipeline(steps=[('std_sc', StandardScaler()),
                            ('pca', pca),
				            ('reg', reg)])

# Estandarizamos el target también
y_transformer = StandardScaler()
inner_estimator = TransformedTargetRegressor(regressor=regr_base, transformer=y_transformer)

## Hiperparametrización

In [ ]:
# Parametros a hiperparametrizar
l_alpha = [10.**k for k in range(-3, 5)]
n_components = list(range(50, 301, 25))
param_grid = {
    'regressor__reg__alpha': l_alpha,
    'regressor__pca__n_components': n_components,
}
# Numero de splits para Cross Validation
n_splits = 6
kf = KFold(n_splits, shuffle=False)
# Hiperparametrizacion
cv_estimator = GridSearchCV(estimator=inner_estimator, 
                            param_grid=param_grid, 
                            cv=kf, 
                            scoring='neg_mean_absolute_error', 
                            return_train_score=True, 
                            n_jobs=5, 
                            verbose=1)

## Entrenamiento y volcado de datos

In [ ]:
# Fit datos 
_ = cv_estimator.fit(data.values, target.values)
print(f"Mejor estimador: {cv_estimator.best_estimator_}")
print(f"Mejor puntuacion - mejor estimador: {cv_estimator.best_score_}")
# Guardar datos en fichero
filename = f'cv_estimator__ridge_{n_components[0]}_{n_components[-1]}_{n_components[1]-n_components[0]}'
dump(cv_estimator, filename)